In [184]:
import json
import os
import pandas as pd
import librosa
import audioread
import numpy as np
import resampy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
import nltk
import warnings
import urllib.request
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from konlpy.tag import Okt
warnings.filterwarnings('ignore')

# 데이터 라벨링

In [251]:
# 일반대화 wav 데이터 불러오기 및 라벨링

os.chdir('/Users/medici/Downloads/json')
text_dir = '/Users/medici/Downloads/json/'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass


d = pd.DataFrame(d)
normal_dir = '/Users/medici/Downloads/일반대화 음성/'
normal_files = os.listdir(normal_dir)
d['wav'] = normal_files
normal= d
normal.rename(columns={0:'txt'},inplace=True)
normal['voice_fishing']=0
normal

,txt,wav,voice_fishing
0,내가 나한테 잘해주면 돼요.혹시 여러분들 자식들가운데?지금 손자 손녀키운다고 먹고 ...,1-converted.ogg,0
1,운명을 바꿔 놓습니다.서울대 음대를 나왔는데요.서울대가 중요한 게 아닙니다.결국 이...,10-converted.ogg,0
2,"네, 감사합니다.네, 감사합니다.서울대 면접없는 판 모국입니다 검사대 여름말씀이신건...",100-converted.ogg,0
3,그러나 인류는 절대 내가 내 말에 절대 라는 용어를 여간 해서는 절대 안 된다는 얘...,11-converted.ogg,0
4,그런데 외국사람이결국은 집 이사 이제 세방을 얻어 는데 그때 50 전세 한 달에 5...,12-converted.ogg,0
...,...,...,...
95,회원 가입을 했거든요 하신다하는 건가요?혹시 방법을 높여야 하는지 잘 몰라서 보는데...,95-converted.ogg,0
96,근데 이제 장소가 이제 구애가 없다고 하시면 저희가 그을지로 고용노동부건물에 있는 ...,96-converted.ogg,0
97,감사합니다.확인 도와드릴 텐데 방송한 기다려 주시겠어요?여기야 기자 감사드리고요.거...,97-converted.ogg,0
98,네 감사합니다.저 인사담당자신청했는데요 그 a니까 아직도 않았어요?네 감사합니다.,98-converted.ogg,0


In [252]:
# 보이스피싱 wav 데이터 불러오기 및 라벨링 

os.chdir('/Users/medici/Downloads/보이스피싱 json')
text_dir = '/Users/medici/Downloads/보이스피싱 json/'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass
d = pd.DataFrame(d)
voice_dir = '/Users/medici/Downloads/보이스피싱 음성/'
voice_files = os.listdir(voice_dir)
d['wav'] = voice_files
voice= d
voice.rename(columns={0:'txt'},inplace=True)
voice['voice_fishing']=1

voice

,txt,wav,voice_fishing
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1-1-converted.ogg,1
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1-2-converted.ogg,1
2,그럼 최근 삼 년 사이에 본인개인정보가 담긴 물품 분실하신 적 전혀 없으십니까?제가...,1-3-converted.ogg,1
3,사실 있는 그대로만 진술하시면 되시는데 현재 계신 곳이 직장 이십니까.통합 불편하시...,1-4-converted.ogg,1
4,일단 이 두 통장은 범죄로 사용돼서 추가적인 피해자가 발생하지 않도록 본인 의사와 ...,1-5-converted.ogg,1
...,...,...,...
77,많은 분들이 연루돼 있는 부분들가운데서 저희가 지금 동시에 수사를 진행하다 보니까 ...,9-4.wav,1
78,이렇게 지금 전화 받아 가지고 있던 말아무도 본인이 지금 왜 보려는 전에 는지 본회...,9-5.wav,1
79,이제 최종 처리를 포함한 공범들하고는 이제 발견된 연관성이 없었기 때문에 일차적인 ...,9-6.wav,1
80,사건 으로 외모되셨으면 확인 한번 부탁드립니다.네 오세요.서울중앙지검에 김진수 아닙...,9-7.wav,1


In [253]:
# 일반대화와 보이스피싱 합치기
df = pd.concat([voice,normal],axis=0)
df.reset_index(drop=True,inplace=True)

# 음성 데이터를 통한 보이스피싱 분류

In [254]:
# wav 데이터 mfcc로 수치화

voice_spam_dir = '/Users/medici/Downloads/보이스피싱 음성/'
normal_dir = '/Users/medici/Downloads/일반대화 음성/'
df['filepath']=''
df['voice_feature']=''


for i in range(len(df)):
    if df.at[i,'voice_fishing']==1:
        df.at[i,'filepath'] = os.path.join(voice_spam_dir, df.at[i,'wav'])
        y, sr = librosa.load(df.at[i,'filepath'], sr=22050)
        y = librosa.util.normalize(y)  # 음량 조정
        mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_mean1 = np.mean(mfccs1, axis=1)
        df.at[i,'voice_feature'] = mfccs_mean1

    else:
        df.at[i,'filepath'] = os.path.join(normal_dir, df.at[i,'wav'])
        y, sr = librosa.load(df.at[i,'filepath'], sr=22050)
        y = librosa.util.normalize(y)  # 음량 조정
        mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_mean1 = np.mean(mfccs1, axis=1)
        df.at[i,'voice_feature'] = mfccs_mean1

In [341]:
# 음성 데이터 학습

df_temp=df.drop(columns=['txt','wav','voice_fishing','filepath'])
X = df_temp
y = df['voice_fishing']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state= 46)
X_train1=list(X_train['voice_feature'])
X_test1=list(X_test['voice_feature'])
y_train1 = list(y_train)
y_test1 = list(y_test)

# 음성 데이터 학습

df_temp=df.drop(columns=['txt','wav','voice_fishing','filepath'])
X = df_temp
y = df['voice_fishing']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state= 18)
X_train1=list(X_train['voice_feature'])
X_test1=list(X_test['voice_feature'])
y_train1 = list(y_train)
y_test1 = list(y_test)


# randomforest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
parameters1 = {
    "n_estimators" : [10, 50, 1000, 2000],
    "max_features" : ["sqrt", "log2"],
    "max_depth" : [2,30, 50,70,100,150,200,300,400]
}
n_iter_search = 10
rf_rgs1 = RandomizedSearchCV(
    rf, 
    param_distributions=parameters1,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
rf_rgs1.fit(X_train1, y_train1)
y_pred_rf1 = rf_rgs1.predict(X_test1)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test1, y_pred_rf1))
print("Recall:", recall_score(y_test1, y_pred_rf1))
print(' ')



# gradient boosting
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
parameters={
    'n_estimators': [1300,1500,1700], 
    'max_depth': [15,20,22],
    'min_samples_leaf': [33,35,40],
    'min_samples_split': [500,700,900],
    'learning_rate': [0.2,0.1],   
}  
n_iter_search = 20
gb_kf_rgs = RandomizedSearchCV(
    gb, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
gb_kf_rgs.fit(X_train1, y_train1)
y_pred_gb1 = gb_kf_rgs.predict(X_test1)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test1, y_pred_gb1))
print("Recall:", recall_score(y_test1, y_pred_gb1))
print(' ')


# xgboost
xgb= XGBClassifier( n_estimators=100,n_jobs=-1)
parameters ={
     "n_estimators":[200,300,400],
     "learning_rate":[0.3, 0.5, 1.0],
     "max_depth" : [6,7],
     "gamma" : [0.1, 0.15],
     "subsample":[0.5, 0.6, 0.7],
     "colsample_bytree":[0.3, 0.5, 1],
}

xgb_gs =GridSearchCV(
    xgb, 
    param_grid=parameters,
    scoring="recall",
    n_jobs=-1, 
)
xgb_gs.fit(X_train1, y_train1)

y_pred_xgb1 = xgb_gs.predict(X_test1)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test1, y_pred_xgb1))
print("Recall:", recall_score(y_test1, y_pred_xgb1))
print(' ')

# Lightgbm
lgbm = LGBMClassifier( random_state=42, n_jobs=-1)

parameters ={
    "n_estimators":[100, 200, 300, 500, 1000],
    "learning_rate":[0.01, 0.1, 0.2, 0.5, 1],

    "max_depth" : [1,3,5,7],
    "min_split_gain" : [0, 0.1, 0.2, 0.4, 0.5],

    "subsample":[0.3, 0.5, 0.7, 0.9],
    "colsample_bytree":[0.3, 0.5, 0.7, 0.8, 0.9],

    "reg_alpha": [0, 0.01, 0.1, 0.5, 1, 10 ],
    "reg_lambda" : [0.01, 0.1, 0.5, 1, 10 ]
}
n_iter_search = 50
lgb_kf_rgs = RandomizedSearchCV(
    lgbm, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)

lgb_kf_rgs.fit(X_train1, y_train1)
y_pred_lgbm = lgb_kf_rgs.predict(X_test1)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test1, y_pred_lgbm))
print("Recall:", recall_score(y_test1, y_pred_lgbm))

RandomForest:
Accuracy: 0.945054945054945
Recall: 0.9090909090909091
 
GradientBoosting:
Accuracy: 0.9340659340659341
Recall: 0.8863636363636364
 
XGBoost:
Accuracy: 0.9230769230769231
Recall: 0.8863636363636364
 
LightGBM:
Accuracy: 0.9560439560439561
Recall: 0.9318181818181818


In [373]:
# 음성 모델 ensemble

ensemble_wav = VotingClassifier(estimators=[('rf', rf_rgs1), ('gb', gb_kf_rgs), ('xgb', xgb_gs), ('lgbm', lgb_kf_rgs)], voting='soft')
ensemble_wav.fit(X_train1, y_train1)
y_pred1 = ensemble_wav.predict(X_test1)

print("ensemble Recall:", recall_score(y_test1, y_pred1))
print("Accuracy:", accuracy_score(y_test1, y_pred1))

ensemble Recall: 0.9024390243902439
Accuracy: 0.9120879120879121


# 텍스트 데이터(TF-IDF)를 통한 보이스피싱 분류

In [257]:
# txt 데이터 TF-IDF로 수치화

temp=list(df['txt'])

df['txt_feature']=''
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=50,lowercase=False) # TF-IDF 상위 50개 추출
for i in range(len(df)):
    df.at[i,'txt_feature'] = vectorizer.fit_transform(temp)[i]

a=pd.DataFrame()
for i in range(len(df)):
    b=pd.DataFrame(df.at[i,'txt_feature'].todense())
    a=pd.concat([a,b],axis=0)
a.reset_index(drop=True,inplace=True)
df =pd.concat([df,a],axis=1)
df=df.drop(columns='txt_feature')

df.head(2)

,txt,wav,voice_fishing,filepath,voice_feature,0,1,2,3,4,...,40,41,42,43,44,45,46,47,48,49
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1-1-converted.ogg,1,/Users/medici/Downloads/보이스피싱 음성/1-1-converted...,"[-329.98514, 126.196266, -32.76896, 11.3803215...",0.0,0.0,0.0,0.0,0.000000,...,0.150224,0.049030,0.287916,0.235100,0.00000,0.000000,0.000000,0.051535,0.0,0.056934
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1-2-converted.ogg,1,/Users/medici/Downloads/보이스피싱 음성/1-2-converted...,"[-316.93973, 157.07793, -23.155636, 8.142115, ...",0.0,0.0,0.0,0.0,0.476662,...,0.000000,0.140148,0.000000,0.224003,0.18568,0.303455,0.169637,0.000000,0.0,0.000000


In [258]:
# TF-IDF 상위 50개 값 확인

vectorizer = TfidfVectorizer(max_features=50, lowercase=False)

voice_tfidf = vectorizer.fit_transform(temp)
feature_names = vectorizer.get_feature_names_out()
indices = np.argsort(vectorizer.idf_)[::-1]
top_n = 50
top_features = [feature_names[i] for i in indices]
top_scores = [vectorizer.idf_[i] for i in indices]
print("중요도 TOP50 :")
for i in range(top_n):
    print(f"{top_features[i]}: {top_scores[i]}")


중요도 TOP50 :
내가: 2.8421903228549468
우리: 2.8082887711792655
본인께서: 2.6541380913520074
연락을: 2.5718999931150353
이거: 2.5206066987274847
저는: 2.4718165345580525
직접: 2.4718165345580525
그러니까: 2.4482860371478585
됩니다: 2.4482860371478585
오늘: 2.42529651892316
본인이: 2.42529651892316
그런데: 2.42529651892316
되어: 2.402823663071101
바로: 2.3382851419335298
가지고: 2.3382851419335298
어떻게: 2.297463147413275
그런: 2.277660520117095
그러면: 2.277660520117095
많이: 2.2582424342599934
본인: 2.2582424342599934
하는: 2.2582424342599934
대해서: 2.2582424342599934
어떤: 2.239194239289299
가지: 2.239194239289299
이게: 2.220502106277147
그냥: 2.220502106277147
다른: 2.20215296760895
있어요: 2.1841344621062717
해서: 2.1841344621062717
일단: 2.1841344621062717
같은: 2.166434885006871
혹시: 2.166434885006871
있습니다: 2.11514159061932
겁니다: 2.11514159061932
근데: 2.0823517677963297
되는: 2.0350988829457837
이런: 2.0350988829457837
하고: 2.0198314108149953
거예요: 1.960990910792062
한번: 1.960990910792062
저희: 1.9054210596372512
저희가: 1.865680730987737
그리고: 1.8527773261518292
이제: 1

In [275]:
# 텍스트 데이터 학습
df_text=df.drop(columns=['txt','wav','filepath','voice_feature'])
X = df_text.drop(columns=['voice_fishing'])
y = df['voice_fishing']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state= 25)
y_train = list(y_train)
y_test = list(y_test)

# randomforest
rf_text = RandomForestClassifier(n_estimators=100, random_state=42)
parameters1 = {
    "n_estimators" : [10, 50, 1000, 2000],
    "max_features" : ["sqrt", "log2"],
    "max_depth" : [2,30, 50,70,100,150,200,300,400]
}
n_iter_search = 10
rf_rgs2 = RandomizedSearchCV(
    rf_text, 
    param_distributions=parameters1,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
rf_rgs2.fit(X_train, y_train)
y_pred_rf2 = rf_rgs2.predict(X_test)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf2))
print("Recall:", recall_score(y_test, y_pred_rf2))
print(' ')



# gradient boosting
gb_text = GradientBoostingClassifier(n_estimators=100, random_state=42)
parameters={
    'n_estimators': [1300,1500,1700], 
    'max_depth': [15,20,22],
    'min_samples_leaf': [33,35,40],
    'min_samples_split': [500,700,900],
    'learning_rate': [0.2,0.1],   
}  
n_iter_search = 20
gb_kf_rgs2 = RandomizedSearchCV(
    gb_text, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
gb_kf_rgs2.fit(X_train, y_train)
y_pred_gb2 = gb_kf_rgs2.predict(X_test)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test, y_pred_gb2))
print("Recall:", recall_score(y_test, y_pred_gb2))
print(' ')


# xgboost
xgb_text= XGBClassifier( n_estimators=100,n_jobs=-1)
parameters ={
     "n_estimators":[200,300,400],
     "learning_rate":[0.3, 0.5, 1.0],
     "max_depth" : [6,7],
     "gamma" : [0.1, 0.15],
     "subsample":[0.5, 0.6, 0.7],
     "colsample_bytree":[0.3, 0.5, 1],
}

xgb_gs2 =GridSearchCV(
    xgb_text, 
    param_grid=parameters,
    scoring="recall",
    n_jobs=-1, 
)
xgb_gs2.fit(X_train, y_train)

y_pred_xgb2 = xgb_gs2.predict(X_test)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb2))
print("Recall:", recall_score(y_test, y_pred_xgb2))
print(' ')

# Lightgbm
lgbm_text = LGBMClassifier( random_state=42, n_jobs=-1)

parameters ={
    "n_estimators":[100, 200, 300, 500, 1000],
    "learning_rate":[0.01, 0.1, 0.2, 0.5, 1],

    "max_depth" : [1,3,5,7],
    "min_split_gain" : [0, 0.1, 0.2, 0.4, 0.5],

    "subsample":[0.3, 0.5, 0.7, 0.9],
    "colsample_bytree":[0.3, 0.5, 0.7, 0.8, 0.9],

    "reg_alpha": [0, 0.01, 0.1, 0.5, 1, 10 ],
    "reg_lambda" : [0.01, 0.1, 0.5, 1, 10 ]
}
n_iter_search = 50
lgb_kf_rgs2 = RandomizedSearchCV(
    lgbm_text, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)

lgb_kf_rgs2.fit(X_train, y_train)
y_pred_lgbm2 = lgb_kf_rgs2.predict(X_test)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test, y_pred_lgbm2))
print("Recall:", recall_score(y_test, y_pred_lgbm2))

RandomForest:
Accuracy: 0.9120879120879121
Recall: 0.8478260869565217
 
GradientBoosting:
Accuracy: 0.945054945054945
Recall: 0.8913043478260869
 
XGBoost:
Accuracy: 0.9120879120879121
Recall: 0.8260869565217391
 
LightGBM:
Accuracy: 0.8901098901098901
Recall: 0.8043478260869565


In [215]:
# 텍스트(TF-IDF) 모델 ensemble
ensemble_txt = VotingClassifier(estimators=[('rf', rf_rgs2), ('gb', gb_kf_rgs2), ('xgb', xgb_gs2), ('lgbm', lgb_kf_rgs2)], voting='soft')
ensemble_txt.fit(X_train, y_train)
y_pred = ensemble_txt.predict(X_test)

print("ensemble Recall:", recall_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

ensemble Recall: 0.9047619047619048
Accuracy: 0.9340659340659341


# 텍스트 데이터(word2vec)로 보이스피싱 분류

In [ ]:
pip install gensim

In [ ]:
!pip install konlpy

In [278]:
# 일반대화와 보이스피싱 합치기
df_sum = pd.concat([voice,normal])
df_sum = df_sum.drop(columns='wav')
df_sum= df_sum.reset_index(drop=True)
df_sum

,txt,voice_fishing
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1
2,그럼 최근 삼 년 사이에 본인개인정보가 담긴 물품 분실하신 적 전혀 없으십니까?제가...,1
3,사실 있는 그대로만 진술하시면 되시는데 현재 계신 곳이 직장 이십니까.통합 불편하시...,1
4,일단 이 두 통장은 범죄로 사용돼서 추가적인 피해자가 발생하지 않도록 본인 의사와 ...,1
...,...,...
177,회원 가입을 했거든요 하신다하는 건가요?혹시 방법을 높여야 하는지 잘 몰라서 보는데...,0
178,근데 이제 장소가 이제 구애가 없다고 하시면 저희가 그을지로 고용노동부건물에 있는 ...,0
179,감사합니다.확인 도와드릴 텐데 방송한 기다려 주시겠어요?여기야 기자 감사드리고요.거...,0
180,네 감사합니다.저 인사담당자신청했는데요 그 a니까 아직도 않았어요?네 감사합니다.,0


In [279]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','요','.','께서','에서','이다','되다']

# 토큰화 
okt = Okt()
tokenized_data = []
for sentence in df_sum['txt']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

# Word2vec 모델 학습
model = Word2Vec(sentences = tokenized_data, window = 5, min_count = 1, workers = 4, sg = 0)
model.wv.vectors.shape

# 각 문장의 벡터 표현 생성
sentence_vectors = []
for text in tokenized_data:
    vector = np.mean([model.wv[word] for word in text], axis=0)
    sentence_vectors.append(vector)


In [287]:
# 텍스트 데이터(work2vec) 모델 학습

labels = list(df_sum['voice_fishing'])
X_train2,X_test2,y_train2,y_test2 = train_test_split(sentence_vectors,labels,test_size = 0.5,random_state= 24)

# randomforest
rf_wv = RandomForestClassifier(n_estimators=100, random_state=42)
parameters1 = {
    "n_estimators" : [10, 50, 1000, 2000],
    "max_features" : ["sqrt", "log2"],
    "max_depth" : [2,30, 50,70,100,150,200,300,400]
}
n_iter_search = 10
rf_rgs3 = RandomizedSearchCV(
    rf_wv, 
    param_distributions=parameters1,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
rf_rgs3.fit(X_train2, y_train2)
y_pred_rf3 = rf_rgs3.predict(X_test2)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test2, y_pred_rf3))
print("Recall:", recall_score(y_test2, y_pred_rf3))
print(' ')



# gradient boosting
gb_wv = GradientBoostingClassifier(n_estimators=100, random_state=42)
parameters={
    'n_estimators': [1300,1500,1700], 
    'max_depth': [15,20,22],
    'min_samples_leaf': [33,35,40],
    'min_samples_split': [500,700,900],
    'learning_rate': [0.2,0.1],   
}  
n_iter_search = 20
gb_kf_rgs3 = RandomizedSearchCV(
    gb_wv, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
gb_kf_rgs3.fit(X_train2, y_train2)
y_pred_gb3 = gb_kf_rgs3.predict(X_test2)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test2, y_pred_gb3))
print("Recall:", recall_score(y_test2, y_pred_gb3))
print(' ')


# xgboost
xgb_wv= XGBClassifier( n_estimators=100,n_jobs=-1)
parameters ={
     "n_estimators":[200,300,400],
     "learning_rate":[0.3, 0.5, 1.0],
     "max_depth" : [6,7],
     "gamma" : [0.1, 0.15],
     "subsample":[0.5, 0.6, 0.7],
     "colsample_bytree":[0.3, 0.5, 1],
}

xgb_gs3 =GridSearchCV(
    xgb_wv, 
    param_grid=parameters,
    scoring="recall",
    n_jobs=-1, 
)
xgb_gs3.fit(X_train2, y_train2)

y_pred_xgb3 = xgb_gs3.predict(X_test2)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test2, y_pred_xgb3))
print("Recall:", recall_score(y_test2, y_pred_xgb3))
print(' ')

# Lightgbm
lgbm_wv = LGBMClassifier( random_state=42, n_jobs=-1)

parameters ={
    "n_estimators":[100, 200, 300, 500, 1000],
    "learning_rate":[0.01, 0.1, 0.2, 0.5, 1],

    "max_depth" : [1,3,5,7],
    "min_split_gain" : [0, 0.1, 0.2, 0.4, 0.5],

    "subsample":[0.3, 0.5, 0.7, 0.9],
    "colsample_bytree":[0.3, 0.5, 0.7, 0.8, 0.9],

    "reg_alpha": [0, 0.01, 0.1, 0.5, 1, 10 ],
    "reg_lambda" : [0.01, 0.1, 0.5, 1, 10 ]
}
n_iter_search = 50
lgb_kf_rgs3 = RandomizedSearchCV(
    lgbm_wv, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)

lgb_kf_rgs3.fit(X_train2, y_train2)
y_pred_lgbm3 = lgb_kf_rgs3.predict(X_test2)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test2, y_pred_lgbm3))
print("Recall:", recall_score(y_test2, y_pred_lgbm3))

RandomForest:
Accuracy: 0.9010989010989011
Recall: 0.8837209302325582
 
GradientBoosting:
Accuracy: 0.9340659340659341
Recall: 0.9534883720930233
 
XGBoost:
Accuracy: 0.9230769230769231
Recall: 0.9069767441860465
 
LightGBM:
Accuracy: 0.9340659340659341
Recall: 0.9302325581395349


In [343]:
# 텍스트(work2vec) 모델 ensemble
ensemble_wv = VotingClassifier(estimators=[('rf', rf_rgs3), ('gb', gb_kf_rgs3), ('xgb', xgb_gs3), ('lgbm', lgb_kf_rgs3)], voting='soft')
ensemble_wv.fit(X_train2, y_train2)
y_pred3 = ensemble_wv.predict(X_test2)

print("ensemble Recall:", recall_score(y_test2, y_pred3))
print("Accuracy:", accuracy_score(y_test2, y_pred3))

ensemble Recall: 0.9302325581395349
Accuracy: 0.9120879120879121


# 음성과 텍스트(TF-IDF, word2vec)을 모두 활용한 보이스피싱 분류

In [362]:
# 음성 feature, 텍스트(TF-IDF,word2vec) feature 결합

voice_data_train = pd.DataFrame(X_train1)
word2vec_data_train = pd.DataFrame(X_train2)
tfidf_data_train = pd.DataFrame(X_train)
voice_data_train.reset_index(drop=True,inplace=True)
word2vec_data_train.reset_index(drop=True,inplace=True)
tfidf_data_train.reset_index(drop=True,inplace=True)
X_train3 = pd.concat([voice_data_train,word2vec_data_train],axis=1)
X_train3 = pd.concat([X_train3,tfidf_data_train],axis=1)

ss = list(range(191))
X_train3.columns=ss
X_test3.columns=ss

w2v = pd.DataFrame(sentence_vectors)
df_all = pd.concat([df,w2v],axis=1)
df_all = df_all.drop(columns=['txt','wav','filepath'])
X_train3 = df_all.drop(columns='voice_fishing')

# 음성과 텍스트 동시에 학습

# randomforest
rf_all = RandomForestClassifier(n_estimators=100, random_state=42)
parameters1 = {
    "n_estimators" : [10, 50, 1000, 2000],
    "max_features" : ["sqrt", "log2"],
    "max_depth" : [2,30, 50,70,100,150,200,300,400]
}
n_iter_search = 10
rf_rgs4 = RandomizedSearchCV(
    rf_all, 
    param_distributions=parameters1,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
rf_rgs4.fit(X_train3, y_train3)
y_pred_rf4 = rf_rgs4.predict(X_test3)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test3, y_pred_rf4))
print("Recall:", recall_score(y_test3, y_pred_rf4))
print(' ')



# gradient boosting
gb_all = GradientBoostingClassifier(n_estimators=100, random_state=42)
parameters={
    'n_estimators': [1300,1500,1700], 
    'max_depth': [15,20,22],
    'min_samples_leaf': [33,35,40],
    'min_samples_split': [500,700,900],
    'learning_rate': [0.2,0.1],   
}  
n_iter_search = 20
gb_kf_rgs4 = RandomizedSearchCV(
    gb_all, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)
gb_kf_rgs4.fit(X_train3, y_train3)
y_pred_gb4 = gb_kf_rgs4.predict(X_test3)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test3, y_pred_gb4))
print("Recall:", recall_score(y_test3, y_pred_gb4))
print(' ')


# xgboost
xgb_all= XGBClassifier( n_estimators=100,n_jobs=-1)
parameters ={
     "n_estimators":[200,300,400],
     "learning_rate":[0.3, 0.5, 1.0],
     "max_depth" : [6,7],
     "gamma" : [0.1, 0.15],
     "subsample":[0.5, 0.6, 0.7],
     "colsample_bytree":[0.3, 0.5, 1],
}

xgb_gs4 =GridSearchCV(
    xgb_all, 
    param_grid=parameters,
    scoring="recall",
    n_jobs=-1, 
)
xgb_gs4.fit(X_train3, y_train3)

y_pred_xgb4 = xgb_gs4.predict(X_test3)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test3, y_pred_xgb4))
print("Recall:", recall_score(y_test3, y_pred_xgb4))
print(' ')

# Lightgbm
lgbm_all = LGBMClassifier( random_state=42, n_jobs=-1)

parameters ={
    "n_estimators":[100, 200, 300, 500, 1000],
    "learning_rate":[0.01, 0.1, 0.2, 0.5, 1],

    "max_depth" : [1,3,5,7],
    "min_split_gain" : [0, 0.1, 0.2, 0.4, 0.5],

    "subsample":[0.3, 0.5, 0.7, 0.9],
    "colsample_bytree":[0.3, 0.5, 0.7, 0.8, 0.9],

    "reg_alpha": [0, 0.01, 0.1, 0.5, 1, 10 ],
    "reg_lambda" : [0.01, 0.1, 0.5, 1, 10 ]
}
n_iter_search = 50
lgb_kf_rgs4 = RandomizedSearchCV(
    lgbm_all, 
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1, 
    random_state=42,
    n_iter = n_iter_search
)

lgb_kf_rgs4.fit(X_train3, y_train3)
y_pred_lgbm4 = lgb_kf_rgs4.predict(X_test3)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test3, y_pred_lgbm4))
print("Recall:", recall_score(y_test3, y_pred_lgbm4))

RandomForest:
Accuracy: 0.9454545454545454
Recall: 0.9629629629629629
 
GradientBoosting:
Accuracy: 0.8727272727272727
Recall: 0.8148148148148148
 
XGBoost:
Accuracy: 0.9454545454545454
Recall: 0.9259259259259259
 
LightGBM:
Accuracy: 0.9090909090909091
Recall: 0.8518518518518519


In [342]:
# 음성과 텍스트 동시 모델 ensemble
ensemble_f = VotingClassifier(estimators=[('rf', rf_rgs4), ('gb', gb_kf_rgs4), ('xgb', xgb_gs4), ('lgbm', lgb_kf_rgs4)], voting='soft')
ensemble_f.fit(X_train3, y_train3)
y_pred3 = ensemble_f.predict(X_test3)

print("ensemble Recall:", recall_score(y_test3, y_pred3))
print("Accuracy:", accuracy_score(y_test3, y_pred3))

ensemble Recall: 0.9318181818181818
Accuracy: 0.967032967032967
